<a href="https://colab.research.google.com/github/syllerim/ai-email-return-automation/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AI Use Cases Assignment 📬

## 📦 Dependencies

In [1]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 May 10 08:25 .
drwxr-xr-x 1 root root 4096 May 10 06:22 ..
drwxr-xr-x 4 root root 4096 May  8 13:38 .config
-rw-r--r-- 1 root root  179 May 10 08:27 .env
drwxr-xr-x 2 root root 4096 May 10 08:25 .ipynb_checkpoints
drwxr-xr-x 1 root root 4096 May  8 13:38 sample_data


In [2]:
# install dependencies

!pip install -q langchain langchain-openai openai python-dotenv

In [3]:
# import dependencies

import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [4]:
# load `.env` variables from `.env` file

load_dotenv(".env")

# access the .env key
# print(os.getenv("OPENAI_API_KEY"))

# or check the entire .env
# !cat .env

True

In [5]:
# create LLM instance

llm = ChatOpenAI(model="gpt-4o", temperature=0)

## 👓 Read and Analyze Email Request

### Pipeline

In [6]:
def process_email(
    llm,
    format_instructions,
    parser,
    customer_email_example,
    email_template,
    reply_template
) -> str:
    """
    Procesa un email de solicitud de devolución de forma automática.

    El flujo incluye tres pasos:

    1. Extrae la información relevante del email usando LangChain:
        - nombre del cliente
        - email del cliente
        - número de pedido
        - identificador del producto
        - si se mencionan imágenes o documentos adjuntos
        - resumen del motivo de la solicitud
        - motivo exacto de la devolución

    2. Evalúa el motivo de la devolución y decide si debe aceptarse o rechazarse,
       en función de las siguientes condiciones:
       ☑️ Aceptar: defecto de fabricación, error en el suministro, producto incompleto
       ❌ Rechazar: daño durante transporte, manipulación indebida, fuera de plazo

    3. Genera una respuesta automática formal y empática utilizando la información extraída y las reglas de negocio.

    Args:
        llm: Instancia de ChatOpenAI ya configurada.
        format_instructions: Instrucciones de formato para el parser.
        parser: Instancia de StructuredOutputParser.
        customer_email_example: Texto del email del cliente a procesar.
        email_template: Prompt para extraer la información del email.
        reply_template: Prompt para generar la respuesta final.

    Returns:
        str: Texto del email de respuesta generado por el modelo.
    """
    # data extraction
    extract_prompt = ChatPromptTemplate.from_template(email_template)
    extract_messages = extract_prompt.format_messages(
        text=customer_email_example,
        format_instructions=format_instructions
    )
    extract_response = llm.invoke(extract_messages)
    extracted_parsed_data = parser.parse(extract_response.content)

    # show the parsed data
    print("\n")
    print(extracted_parsed_data)

    # answer generation
    reply_prompt = ChatPromptTemplate.from_template(reply_template)
    reply_messages = reply_prompt.format_messages(**extracted_parsed_data)

    # this is the same as doing it explicitly, because I am passing the same dict
    # reply_messages = reply_prompt.format_messages(
    #     customer_name=parsed_output["customer_name"],
    #     customer_email=parsed_output["customer_email"],
    #     order_number=parsed_output["order_number"],
    #     product_identifier=parsed_output["product_identifier"],
    #     email_attachments=parsed_output["email_attachments"],
    #     email_subject=parsed_output["email_subject"],
    #     email_reason_details=parsed_output["email_reason_details"]
    # )

    final_reply_email = llm.invoke(reply_messages)

    return final_reply_email.content.strip()

### Template definitions

In [7]:
# create the prompt to extract the info from email. Format it by injecting the email text and formatting instructions

email_template = """Para siguiente texto correo electrónico de cliente, extrae siguiente información:

customer_name: Nombre del cliente que firma el correo.
customer_email: Email del cliente.
order_number: Número de pedido mencionado en el email.
product_identifier: Identificador del producto.
email_attachments: ¿Se mencionan imágenes o documentos adjuntos? Responder 'Sí' o 'No'.
email_subject: Resumen breve del motivo de la solicitud.
email_reason_details: ¿Cuál es el motivo exacto de la devolución según las categorías definidas?
- Defecto de fabricación confirmado
- Error en el suministro
- Producto incompleto o faltante
- Daño durante transporte
- Manipulación indebida
- Fuera de plazo

Original email: {text}

{format_instructions}
"""

In [8]:
# build the service reply based on extracted data

reply_template = """Redacta una respuesta formal, clara y empática por parte del equipo de Componentes Intergalácticos Industriales S.A. a un cliente que ha enviado una solicitud de devolución.

Escríbela directamente como si fuera un email al cliente.

Ten en cuenta esta información:

- Nombre del cliente: {customer_name}
- Email del cliente: {customer_email}
- Número de pedido: {order_number}
- Identificador del producto: {product_identifier}
- Se mencionan imágenes o documentos adjuntos: {email_attachments}
- Resumen del motivo de la solicitud: {email_subject}

Si el motivo de devolución es uno de estos:
✅ "Defecto de fabricación confirmado", "Error en el suministro", "Producto incompleto o faltante", entonces la respuesta debe aceptar la devolución.

Si el motivo es uno de estos:
❌ "Daño durante transporte", "Manipulación indebida", "Fuera de plazo", la respuesta debe rechazar la devolución amablemente explicando el porqué.

Firmal atentamente de nuestra parte.
"""

### Extract Data and Processs it

In [9]:
# define fields to extract from the email

customer_name_schema = ResponseSchema(
    name="customer_name", description="Nombre del cliente que envía el email."
)
customer_email_schema = ResponseSchema(
    name="customer_email", description="Email del cliente."
)
order_number_schema = ResponseSchema(
    name="order_number", description="Número de pedido mencionado en el email."
)
product_identifier_schema = ResponseSchema(
    name="product_identifier", description="Identificador del producto."
)
email_attachments_schema = ResponseSchema(
    name="email_attachments", description="¿Se mencionan imágenes o documentos adjuntos? Responder 'Sí' o 'No'."
)
email_subject_schema = ResponseSchema(
    name="email_subject", description="Resumen breve del motivo de la solicitud."
)
email_reason_details_schema = ResponseSchema(
    name="email_reason_details", description="¿Cuál es el motivo exacto de la devolución según las categorías definidas?."
)
schemas = [
    customer_name_schema,
    customer_email_schema,
    order_number_schema,
    product_identifier_schema,
    email_attachments_schema,
    email_subject_schema,
    email_reason_details_schema,
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

### Execution

**⛔️ Example of negative reply**

In [10]:
# example of original customer email

customer_email_example = """
Asunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA

Estimado equipo de Componentes Intergalácticos Industriales S.A.,

Me pongo en contacto con ustedes como cliente reciente para comunicar una incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran envergadura: la construcción de la Estrella de la Muerte.

Lamentablemente, al recibir el envío, observamos que varios de los condensadores presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y consultar con el piloto de carga, todo indica que la mercancía sufrió una caída durante el transporte interestelar.

Dado que estos componentes son críticos para la activación del núcleo central del sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo inmediato de las unidades defectuosas, así como una revisión de los protocolos de embalaje y transporte para evitar que algo así vuelva a ocurrir.

Adjunto imágenes del estado de los condensadores y el albarán de entrega sellado por nuestro droide de recepción.

Agradezco de antemano su pronta atención a este asunto. Quedamos a la espera de su respuesta para coordinar el reemplazo.

Atentamente,
Darth Márquez
Departamento de Ingeniería Imperial
Sector de Proyectos Especiales
Contacto: dmarquez@imperiumgalactic.net
Holofono: +34 9X9 123 456”
"""

In [11]:
customer_service_reply = process_email(
    llm,
    format_instructions,
    parser,
    customer_email_example,
    email_template,
    reply_template
)



{'customer_name': 'Darth Márquez', 'customer_email': 'dmarquez@imperiumgalactic.net', 'order_number': 'D347-STELLA', 'product_identifier': 'FX-88', 'email_attachments': 'Sí', 'email_subject': 'Solicitud de reemplazo por daños en transporte', 'email_reason_details': 'Daño durante transporte'}


In [12]:
customer_service_reply

'Asunto: Respuesta a su Solicitud de Devolución - Pedido D347-STELLA\n\nEstimado Darth Márquez,\n\nEsperamos que este mensaje le encuentre bien. Agradecemos que se haya puesto en contacto con nosotros y nos haya proporcionado la información detallada sobre su solicitud de devolución del producto FX-88, correspondiente al pedido número D347-STELLA.\n\nHemos revisado cuidadosamente su solicitud y los documentos adjuntos que nos ha enviado. Entendemos que el motivo de su solicitud es el daño sufrido por el producto durante el transporte. Lamentablemente, según nuestra política de devoluciones, no podemos aceptar devoluciones por daños ocurridos durante el transporte, ya que estos incidentes deben ser gestionados directamente con la empresa de transporte responsable.\n\nSin embargo, queremos ofrecerle nuestro apoyo en este proceso. Le recomendamos que se comunique con la empresa de transporte para presentar una reclamación por los daños sufridos. Si necesita cualquier tipo de documentación

**➕ Example of positive reply**

In [13]:
# example of original customer email

customer_email_defecto_fabricacion = """
Asunto: Solicitud de reemplazo por defecto de fabricación – Pedido #X102-MDA458

Estimado equipo de Componentes Intergalácticos Industriales S.A.,

Me pongo en contacto con ustedes desde el Departamento de MarviWorld para notificar un problema relacionado con el pedido #X102-MDA458, correspondiente a un lote de cristales de calibración quántica KJU-7, adquiridos recientemente para la nueva generación de sistemas de navegación inercial de rovers.

Al realizar las primeras pruebas de integración, detectamos fallos recurrentes en el rendimiento de algunos sensores, específicamente en el sistema de redundancia aeroespacial. Tras un análisis técnico interno, concluimos que se trata de un defecto de fabricación, ya que los fallos son consistentes y no atribuibles al montaje o a condiciones externas.

Dado que estos componentes son esenciales para garantizar la maniobrabilidad y seguridad de nuestros rovers, solicitamos el reemplazo urgente de los lotes defectuosos, así como una evaluación de calidad por parte de su equipo técnico para evitar recurrencias en futuros lotes.

Adjuntamos el informe técnico detallado junto con los números de serie de las unidades afectadas.

Agradecemos su pronta atención y quedamos a la espera de instrucciones para coordinar el envío de las piezas de reemplazo.

Atentamente,
Comandante T. Voss
Departamento de Ingeniería de MarviWorld
Contacto: tvoss@marviworld.net
Holofono: +34 9X9 987 451
"""


In [14]:
customer_service_reply = process_email(
    llm,
    format_instructions,
    parser,
    customer_email_defecto_fabricacion,
    email_template,
    reply_template
)



{'customer_name': 'Comandante T. Voss', 'customer_email': 'tvoss@marviworld.net', 'order_number': 'X102-MDA458', 'product_identifier': 'KJU-7', 'email_attachments': 'Sí', 'email_subject': 'Solicitud de reemplazo por defecto de fabricación', 'email_reason_details': 'Defecto de fabricación confirmado'}


In [15]:
customer_service_reply

'Asunto: Confirmación de Devolución para el Pedido X102-MDA458\n\nEstimado Comandante T. Voss,\n\nEsperamos que este mensaje le encuentre bien. Nos dirigimos a usted en respuesta a su solicitud de devolución relacionada con el pedido número X102-MDA458, específicamente sobre el producto identificado como KJU-7.\n\nEn primer lugar, queremos agradecerle por proporcionarnos las imágenes y documentos adjuntos que nos han permitido evaluar su caso de manera detallada. Tras una revisión exhaustiva, hemos confirmado que el producto presenta un defecto de fabricación.\n\nPor lo tanto, nos complace informarle que su solicitud de devolución ha sido aceptada. Procederemos a gestionar el reemplazo del producto defectuoso a la mayor brevedad posible. Nuestro equipo de logística se pondrá en contacto con usted para coordinar los detalles del envío del nuevo producto y la recogida del artículo defectuoso.\n\nLamentamos sinceramente cualquier inconveniente que este problema haya podido causarle y agra